## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [28]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
usuario_mysql = os.getenv('usuario_mysql')
senha_mysql = os.getenv('senha_mysql')
servidor_mysql = os.getenv('servidor_mysql')

print(f"usuario_mysql: {usuario_mysql}")
print(f"senha_mysql: {senha_mysql}")
print(f"servidor_mysql: {servidor_mysql}")


usuario_mysql: henrique
senha_mysql: 12345
servidor_mysql: localhost


In [30]:
import mysql.connector

cnx = mysql.connector.connect(
    host=servidor_mysql,
    user=usuario_mysql,
    password=senha_mysql
)

print(cnx)

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [4]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [5]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [6]:
cursor.execute("show databases;")

for db in cursor:
    print(db)


('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [7]:
import pandas as pd

df_livros = pd.read_csv('/home/henrique/pipeline-python-mongo-mysql/data/tabela_livros.csv')
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,6695c367b579d39baec6d9b7,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
1,6695c367b579d39baec6d9c9,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
2,6695c367b579d39baec6d9d7,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
3,6695c367b579d39baec6d9d9,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55
4,6695c367b579d39baec6d9dc,Iniciando em programação,livros,61.73,0.000000,2021-02-01,Beatriz Moraes,SP,4,cartao_credito,1,-22.19,-48.79


In [8]:
df_livros.columns # colunas

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [9]:
df_livros.shape

(1483, 13)

In [10]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
        id VARCHAR(100),
        Produto VARCHAR(100),
        Categoria_Produto VARCHAR(100),
        Preco FLOAT(10,2),
        Frete FLOAT(10,2),
        Data_Compra DATE,
        Vendedor VARCHAR(100),
        Local_Compra VARCHAR(100),
        Avaliacao_Compra INT,
        Tipo_Pagamento VARCHAR(100),
        Qntd_Parcelas INT,
        Latitude FLOAT(10,2),
        Longitude FLOAT(10,2),
               
        PRIMARY KEY (id)
    );
""")

**Selecionando a base de dados para verificar a tabela criada**

In [11]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
    print(tb)

('tb_livros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [12]:
for i, row in df_livros.iterrows():
    print(tuple(row))

('6695c367b579d39baec6d9b7', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('6695c367b579d39baec6d9c9', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('6695c367b579d39baec6d9d7', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('6695c367b579d39baec6d9d9', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('6695c367b579d39baec6d9dc', 'Iniciando em programação', 'livros', 61.73, 0.0, '2021-02-01', 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('6695c367b579d39baec6d9df', 'Dashboards com Power BI', 'livros', 57.63, 9.3105595123, '2022-02-01', 'Bianca Santos', 'SP', 3, 'cartao_credito', 1, -22.19, -48.79)
('6695c367b579d39ba

In [13]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

[('6695c367b579d39baec6d9b7',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('6695c367b579d39baec6d9c9',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('6695c367b579d39baec6d9d7',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('6695c367b579d39baec6d9d9',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('6695c367b579d39baec6d9dc',
  'Iniciando em programação',
  'livros',
  61.73,
  0.0,
  '2021-02-01',
  'Beatriz Moraes',
  'SP',
  4,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('6695c367b579d39baec6d9df',
  'Dashboards com Power

In [14]:
try:
    sql = "INSERT INTO dbprodutos.tb_livros VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

    cursor.executemany(sql, lista_dados)

    cnx.commit()
except:
    print("Dados já inseridos")

In [15]:
print(cursor.rowcount, "dados inseridos")

1483 dados inseridos


## Visualizando os dados inseridos

In [16]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros")

for row in cursor:
    print(row)

('6695c367b579d39baec6d9b7', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('6695c367b579d39baec6d9c9', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('6695c367b579d39baec6d9d7', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('6695c367b579d39baec6d9d9', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('6695c367b579d39baec6d9dc', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('6695c367b579d39baec6d9df', 'Dashboards com Power BI', 'livros', 57.63, 9.31, datetime.date(2022, 2, 1), 'Bianca Santos', 'SP', 3, 'cartao_

In [17]:
cursor.close()

True

In [18]:
cnx.close()